In [1]:
%matplotlib qt

import hyperspy.api as hs
import numpy as np
import os
from pathlib import Path
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from math import sqrt
import datetime as dt
import mib2hspy as m2h


In [2]:
def wavelength(V, m0=9.1093837015*1e-31, e=1.60217662*1e-19, h=6.62607004*1e-34 , c=299792458):
    """
    Return the wavelength of an accelerated electron in [Å]
    
    Arguments
    ---------
    m0 : float, Rest mass of electron [kg]
    e : float, Elementary charge of electron [C]
    h : float, Planck' constant [m^2 kg/s]
    c : float, Speed of light in vacuum [m/s]
    """
    return h / sqrt( 2 * m0 * e * V * ( 1.0 + ( e*V / ( 2*m0*c**2 ) ) ) ) * 1E10

def calculate_camera_length(scale, pixel_size, wavelength):
    """
    Return the camera length in cm
    
    Arguments
    ---------
    scale : float, scale of pixels [1/nm/px]
    pixel_size: float, Physical size of pixels on detector [um].
    wavelength : float, Wavelength of electrons [Å]
    """
    
    wavelength = float(wavelength) / 10 #convert to nm
    pixel_size = pixel_size * 1E3 #convert to nm
    return pixel_size / wavelength / scale * 1E-9

def get_calibration_from_GMS(signal, label=''):
    """
    Get calibration data from a hyperspy signal with GMS-like metadata and attach a label.
    
    Parameters
    ----------
    signal: an image with GMS metadata.
    label: str. A string to use as label for the calibration.
    
    Returns
    -------
    calibrations: a pandas dataframe with calibrations and relevant metadata
    """
    
    scale_x, scale_y = signal.axes_manager['x'].scale, signal.axes_manager['y'].scale
    units_x, units_y = signal.axes_manager['x'].units, signal.axes_manager['y'].units
    
    date = signal.original_metadata.ImageList.TagGroup0.ImageTags.DataBar.Acquisition_Date
    microscope = signal.original_metadata.ImageList.TagGroup0.ImageTags.Session_Info.Microscope
    camera = signal.original_metadata.ImageList.TagGroup0.ImageTags.Acquisition.Device.Name
    
    if len(microscope)==0:
        microscope = None
    microscope_info = signal.original_metadata.ImageList.TagGroup0.ImageTags.Microscope_Info
    high_tension = int(microscope_info.Voltage)/1000
    mode = microscope_info.Imaging_Mode
    nominal_mag = round(float(microscope_info.Indicated_Magnification), ndigits=0)
    if mode == 'DIFF':
        nominal_mag = nominal_mag / 10 #convert mm to cm
        device_info = signal.original_metadata.ImageList.TagGroup0.ImageTags.Acquisition.Device
        pixels = device_info.Active_Size_pixels[0]
        pixel_size = device_info.CCD.Pixel_Size_um[0]
        actual_mag = round(float(calculate_camera_length(scale_x, pixel_size, wavelength(high_tension*1000)))*100, ndigits=2)
        scale_deg = np.arcsin((pixel_size*1E-6) / (actual_mag*1E-2))*180/np.pi
        width = scale_deg * pixels
        headers = ['Label', 'Mode', 'CL (cm)', 'Act. CL (cm)', 'Scale (1/nm/px)', 'Scale (deg/px)', 'Width (deg)', 'HT (kV)', 'Date', 'Microscope', 'Camera']
        calibration = [label, mode, nominal_mag, actual_mag, scale_x, scale_deg, width, high_tension, date, microscope, camera]
    else:
        actual_mag = round(float(microscope_info.Actual_Magnification), ndigits=0)
        headers = ['Label', 'Mode', 'Mag.', 'Act. Mag.', 'Scale', 'Units','HT (kV)', 'Date', 'Microscope', 'Camera']
        calibration = [label, mode, nominal_mag, actual_mag, scale_x, units_x, high_tension, date, microscope, camera]
    return pd.DataFrame([calibration], columns = headers)

def get_calibration_from_MERLIN(signal, label, pixel_size=(55, 55), pixels = (256, 256)):
    scale_x, scale_y = signal.axes_manager[0].scale, signal.axes_manager[0].scale
    units_x, units_y = signal.axes_manager[0].units, signal.axes_manager[0].units
    
    try:
        date = dt.datetime.fromisoformat(signal.original_metadata.Session.Date).date().strftime('%d/%m/%Y')
    except AttributeError:
        date = None
    microscope = '2100F'
    camera = 'Merlin'
    
    if len(microscope)==0:
        microscope = None
    microscope_info = signal.original_metadata.Acquisition_instrument.TEM
    try:
        high_tension = int(microscope_info.acceleration_voltage)/1000
    except TypeError:
        high_tension = microscope_info.acceleration_voltage
    mode = microscope_info.mode
    if mode == 'SAEDP':
        
        scale_x = scale_x / 10 #congert from 1/Å to 1/nm
        scale_y = scale_y / 10 #convert from 1/Å to 1/nm
        try:
            nominal_cameralength = round(float(microscope_info.nominal_cameralength), ndigits=0)
        except TypeError:
            nominal_cameralength = microscope_info.nominal_cameralength
        pixels = pixels[0]
        pixel_size = pixel_size[0]
        actual_cameralength = round(float(calculate_camera_length(scale_x, pixel_size, wavelength(high_tension*1000))), ndigits=2)
        scale_deg = np.arcsin((pixel_size*1E-6) / (actual_cameralength*1E-2))*180/np.pi
        width = scale_deg * pixels
        headers = ['Label', 'Mode', 'CL (cm)', 'Act. CL (cm)', 'Scale (1/nm/px)', 'Scale (deg/px)', 'Width (deg)', 'HT (kV)', 'Date', 'Microscope', 'Camera']
        calibration = [label, mode, nominal_cameralength, actual_cameralength, scale_x, scale_deg, width, high_tension, date, microscope, camera]
    else:
        try:
            nominal_magnification = round(float(microscope_info.nominal_magnification), ndigits=0)
        except TypeError:
            nominal_magnification = microscope_info.nominal_magnification
        actual_magnification = round(pixel_size[0]/scale_x, ndigits=0)
        headers = ['Label', 'Mode', 'Mag.', 'Act. Mag.', 'Scale', 'Units','HT (kV)', 'Date', 'Microscope', 'Camera']
        calibration = [label, mode, nominal_magnification, actual_magnification, scale_x, units_x, high_tension, date, microscope, camera]
    return pd.DataFrame([calibration], columns = headers)

In [3]:
calibrations = pd.DataFrame()

In [4]:
#filetypes = ['.dm4']
#directory = Path(r'./2100F/Ultrascan/2020_11_20_TED_PELLA_673/')
filetypes = ['.hspy']
directory = Path(r'./2100F/Merlin/2020_11_21_TEDPELLA673/')
for filename in os.listdir(directory):
    if any([filename.endswith(filetype) for filetype in filetypes]):
        signal = hs.load(str(directory/filename), lazy=True)
        try:
            if '.dm' in filename:
                calibration = get_calibration_from_GMS(signal, label=filename.split('_')[-1].split('.')[0])
            elif '.hspy' in filename:
                calibration = get_calibration_from_MERLIN(signal, label=filename.split('.')[0])
            else:
                print('Did not load {}'.format(directory/filename))
        except Exception as e:
            print(e)
            print(signal)
            print(signal.metadata)
            print(signal.original_metadata)
        else:
            calibrations = pd.concat([calibrations, calibration], axis=0, ignore_index=True, join='outer')
calibrations.drop_duplicates(inplace=True)
print(calibrations)
#print(tabulate(calibrations.sort_values(by=['Mag.']), headers = calibrations.columns, floatfmt=['.0f', 's', '.0f', '.2f', '.3g', 's', '.3g', 's', '.0f', 's'], showindex=False))

'DictionaryTreeBrowser' object has no attribute 'acceleration_voltage'
<LazyElectronDiffraction2D, title: , dimensions: (256, 256|256, 256)>
├── General
│   └── title = 
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

├── Acquisition_instrument
│   └── TEM
│       ├── Parameters
│       │   ├── acquisition_date
│       │   │   └── Units = 
│       │   ├── alpha
│       │   │   ├── Units = 
│       │   │   └── Value = 5
│       │   ├── camera_length
│       │   │   ├── Actual value = nan
│       │   │   ├── Nominal value = 8.0
│       │   │   └── Units = cm
│       │   ├── condenser_aperture
│       │   │   ├── Actual value = nan
│       │   │   ├── Nominal value = nan
│       │   │   └── Units = um
│       │   ├── convergence_angle
│       │   │   ├── Actual value = nan
│       │   │   ├── Nominal value = nan
│       │   │   └── Units = mrad
│       │   ├── ht
│       │   │   ├── Units = V
│       │   │   └── Value = 200000.0
│       │   ├── magnification


In [5]:
calibrations

,Label,Mode,CL (cm),Act. CL (cm),Scale (1/nm/px),Scale (deg/px),Width (deg),HT (kV),Date,Microscope,Camera,Mag.,Act. Mag.,Scale,Units
0,CL10cm,SAEDP,10.0,19.64,0.001116,0.016045,4.107559,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
1,CL12cm,SAEDP,12.0,22.95,0.000955,0.013731,3.515140,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
2,CL15cm,SAEDP,15.0,28.16,0.000779,0.011191,2.864789,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
3,CL20cm,SAEDP,20.0,36.14,0.000607,0.008720,2.232221,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
4,CL25cm,SAEDP,25.0,44.75,0.000490,0.007042,1.802736,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
5,CL30cm,SAEDP,30.0,53.44,0.000410,0.005897,1.509589,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
6,CL40cm,SAEDP,40.0,70.33,0.000312,0.004481,1.147056,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
7,CL50cm,SAEDP,50.0,88.71,0.000247,0.003552,0.909395,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
8,CL60cm,SAEDP,60.0,107.28,0.000204,0.002937,0.751980,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
9,CL80cm,SAEDP,80.0,140.66,0.000156,0.002240,0.573528,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN


In [6]:
calibrations[calibrations['Camera']=='Merlin']

,Label,Mode,CL (cm),Act. CL (cm),Scale (1/nm/px),Scale (deg/px),Width (deg),HT (kV),Date,Microscope,Camera,Mag.,Act. Mag.,Scale,Units
0,CL10cm,SAEDP,10.0,19.64,0.001116,0.016045,4.107559,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
1,CL12cm,SAEDP,12.0,22.95,0.000955,0.013731,3.515140,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
2,CL15cm,SAEDP,15.0,28.16,0.000779,0.011191,2.864789,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
3,CL20cm,SAEDP,20.0,36.14,0.000607,0.008720,2.232221,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
4,CL25cm,SAEDP,25.0,44.75,0.000490,0.007042,1.802736,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
5,CL30cm,SAEDP,30.0,53.44,0.000410,0.005897,1.509589,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
6,CL40cm,SAEDP,40.0,70.33,0.000312,0.004481,1.147056,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
7,CL50cm,SAEDP,50.0,88.71,0.000247,0.003552,0.909395,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
8,CL60cm,SAEDP,60.0,107.28,0.000204,0.002937,0.751980,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN
9,CL80cm,SAEDP,80.0,140.66,0.000156,0.002240,0.573528,200,23/11/2020,2100F,Merlin,NaN,NaN,NaN,NaN


In [7]:
calibrations.sort_values(by='CL (cm)', inplace=True)
CL_values = calibrations[['Camera', 'CL (cm)', 'Act. CL (cm)', 'Scale (1/nm/px)', 'Scale (deg/px)', 'Width (deg)']]
CL_values = CL_values[CL_values['Camera']=='Merlin']
print(tabulate(CL_values, headers=CL_values.columns, floatfmt = ['.0f', 's', '.0f', '.2f', '.3g', '.3g', '.2f']))

    Camera      CL (cm)    Act. CL (cm)    Scale (1/nm/px)    Scale (deg/px)    Width (deg)
--  --------  ---------  --------------  -----------------  ----------------  -------------
10  Merlin            8           16.20           0.00135            0.0195            4.98
 0  Merlin           10           19.64           0.00112            0.016             4.11
 1  Merlin           12           22.95           0.000955           0.0137            3.52
 2  Merlin           15           28.16           0.000779           0.0112            2.86
 3  Merlin           20           36.14           0.000607           0.00872           2.23
 4  Merlin           25           44.75           0.00049            0.00704           1.80
 5  Merlin           30           53.44           0.00041            0.0059            1.51
 6  Merlin           40           70.33           0.000312           0.00448           1.15
 7  Merlin           50           88.71           0.000247           0.00355    

## Show cameralength calibrations

In [ ]:
calibrations.sort_values(by='CL (cm)', inplace=True)
sns.lineplot(x='CL (cm)', y = 'Act. CL (cm)', hue='Camera', style='Microscope', markers=True, data=calibrations)

In [ ]:
try:
    print(tabulate(calibrations.sort_values(by=['Mag.']), headers = calibrations.columns, floatfmt=['.0f', 's', '.0f', '.2f', '.3g', 's', '.3g', 's', '.0f', 's'], showindex=False))
except KeyError:
    print(tabulate(calibrations.sort_values(by=['CL (cm)']), headers = calibrations.columns, floatfmt=['.0f', 's', '.0f', '.2f', '.3g', '.3g', '.3g', '.0f', 's'], showindex=False))
    sns.lineplot(x='CL (cm)', y='Act. CL (cm)', data=dataframe, markers="X", style='Microscope', hue='HT (kV)')
fig = plt.gcf()
#plt.savefig()
#print(dataframe)

#print(tabulate(calibrations, headers=['Label', 'Nominal Mag.', 'Actual Mag.', 'Scale X', 'Units', 'Scale Y', 'Units', 'HT (V)', 'Date']))

In [ ]:
sns.catplot

In [ ]:
dataframe.plot(x='Mag.', y='')

In [ ]:
dataframe.sort_values(by=['Nominal Mag.'])

In [ ]:
signal.original_metadata.ImageList.TagGroup0.ImageTags